# Practica 4

## Ejercicio 1

### a

Queremos solucionar el problema de los case sensitive usando comparadores, esto significa modificar las funciones shuffle y sort. La salida de los mappers es <first_letter, 1> lo que nosotros tenemos que hacer es solucionar que "A" y "a" vayan al mismo nodo y se ordenen juntas, podemos modificar los cmp usando alguna función estilo to lower_case(). 

In [2]:
import datetime,math,re
import random
import sys
sys.path.append('../..')

from MRE import Job


root_path = './' #Simplemente para poder usar rootpaths
input_path = root_path + "input/"
output_path = root_path + "output/"
input_dir = input_path + "eje1/"
output_dir = output_path + "eje1/"


def map(key, values, context):
    words = values.split()
    for w in words:
        context.write(w[0], 1)
        
def freduce(key, values, context):
    c=0
    for v in values:
        c=c+1
    context.write(key, c)
    
def fShuffleCmp(aKey, anotherKey):
     if (aKey.lower() == anotherKey.lower()):
         return 0
     elif (aKey.lower() < anotherKey.lower()):
         return -1
     else: 
         return 1

job = Job(input_dir,output_dir,map,freduce)
job.setShuffleCmp(fShuffleCmp)
job.waitForCompletion()


True

### b

Se van a ejecutar tantos reducers como caracteres (en lower() indiferente si es "A" o "a") encuentre el map. No termino de entenderla, supongo que se podrían crear claves intermedias que hagan un random entre 1 y 2 pero luego los cmp nose como afectarían ***preguntar!***

## Ejercicio 2

Es muy util para tareas relacionadas a conteos, agregaciones, operaciones tipo base de datos, frecuencias. Con los mappers se debe prestar atención principalmente a que las claves intermedias emitidas sea coherentes con la lógica implementada en los diferentes cmp.

## Ejercicio 3

En principio el combiner no ayuda en nada, inclusive puede generar problemas, mi entendimiento es el siguiente, un combiner trabaja con la salida de los mappers, en este caso tenemos dos mappers, combinar las diferentes salidas puede mezclar info, tal vez podría servir para las cajas de ahorro, pero que es exactamente lo que combinaríamos, al fin y al cabo agrupar los valores de un cliente en una lista no tiene mucho sentido, porque ya es lo que le llega al reducer.

## Ejercicio 4

Queremos determinar la mediana de una serie de valores, en principio la tarea map lo que tiene que hacer es extraer el valor del data-set website y devolverlo, tendríamos que tener un solo reducer que pueda ver toda la lista en principio, pero seria interesante que el sort ya le de la lista ordenada de menor a mayor, lo que podemos hacer es emitir claves estilo 1x donde x es el numero que nos interesa, entonces en el shuffle simplemente miramos el 1, y luego en el sort ya si las ordenamos como se debe, luego miramos los value en el propio reducer, luego esta la lógica para determinar la mediana. Para el calcula de la ventana voy a replicar un lista en el reduce y listo ***preguntar***!

In [ ]:
import datetime,math,re
import random
import sys
sys.path.append('../..')

from MRE import Job


root_path = './' #Simplemente para poder usar rootpaths
input_path = root_path + "input/"
output_path = root_path + "output/"
input_dir = input_path + "eje2/"
output_dir = output_path + "eje2/"

def fmap(key, value, context):
    data = value.split('\t')
    time = data[1]
    context.write((1, time), 1)

def fShuffleCmp(aKey, anotherKey):
    if (aKey[0] == anotherKey[0]):
        return 0
    elif (aKey[0] < anotherKey[0]):
        return -1
    else:
        return 1

def fSortCmp(aKey, anotherKey):
    if (aKey[1] == anotherKey[1]):
        return 0
    elif (aKey[1] < anotherKey[1]):
        return -1
    else:
        return 1

# No se me ocurre como implementar el reducer.

## Ejercicio 5

Okey, dado que la solución se debe realizar sin utilizar parámetros entonces lo que podemos hacer es tener dos inputs con dos mappers, uno se encarga de simplemente pasar los valores al reducer y trabaja sobre un solo archivo, y el otro se encarga de enviar los diferentes valores para resolver el problema, tenemos que tener una sola tupla de entrada, es importante que los valores de las variables sean lo primero que el reducer procese para ya tenerlas, por lo que debemos modificar el shuffle y el sort. Preguntar

In [ ]:
#Preguntar eje 5

## Ejercicio 6

